# Gabor Thesis calculations to obtain the sensor detection

### 

In [1]:
import numpy as np
from scipy.integrate import quad

# Constants
planck_constant = 6.62607004e-34  # m^2 kg/s
speed_of_light = 299792458  # m/s
boltzmann_constant = 1.38064852e-23 #m^2 kg s-2 K-1


## Functions 



In [2]:
# from Gabor Thesis and Accuracy Performance of Star Trackers (Liebe 2002)
def radiative_spectral_emittance(wavelength_, temperature_):
    """
    The radiation from a black body at a given
    wavelength and temperature is given by
    :param wavelength_: in m
    :param temperature_: in kelvin
    :return:
    """

    I_star = (2 * np.pi * planck_constant * speed_of_light ** 2) / \
             (wavelength_ ** 5 * (np.exp(planck_constant * speed_of_light / (wavelength_ * boltzmann_constant * temperature_)) - 1))
    return I_star


def radiant_flux_perratioflux_integral(lambda_interval_bottom_, lambda_interval_top_, temperature_):
    """
    Equation 2.7 from Gabor thesis
    :param lambda_interval_bottom_: integral limit
    :param lambda_interval_top_: integral limit
    :param temperature_:
    :return:
    """
    F_star_fluxratio = quad(radiative_spectral_emittance, lambda_interval_bottom_, lambda_interval_top_,
                            args=temperature_)
    return F_star_fluxratio


def radiant_flux_calculator(flux_sun_, lambda_interval_bottom_, lambda_interval_top_, temperature_):
    """
    Total power in Watts/m^2
    :param flux_sun_:
    :param lambda_interval_bottom_:
    :param lambda_interval_top_:
    :param temperature_:
    :return:
    """
    flux_star_fluxratio = radiant_flux_perratioflux_integral(lambda_interval_bottom_, lambda_interval_top_, temperature_)
    radiant_flux = np.sqrt(flux_sun_ * flux_star_fluxratio[0])
    radiant_flux_error = ((1/2)*np.sqrt(flux_sun_ * flux_star_fluxratio[0])/(flux_star_fluxratio[0]))*flux_star_fluxratio[1]
    return radiant_flux, radiant_flux_error


def planck_einstein_relation(wavelength_):
    """
    This function calculates the energy of a photon in a specific wavelength
    :param wavelength_ in meter
    :return: energy_photon in joule
    """
    energy_photon = (planck_constant * speed_of_light) / wavelength_
    return energy_photon


def photon_spectral_emittance(radiant_flux_, wavelength_):
    """
    Function for the luminosity of a star. [photons/(s*mm^2)]
    :param radiant_flux_:
    :param wavelength_:
    :return:
    """

    L_star = radiant_flux_/planck_einstein_relation(wavelength_)
    return L_star


def sensor_photon_irradiance(wavelength_, radiant_flux_, quantum_efficiency_):
    L_star = photon_spectral_emittance(radiant_flux_, wavelength_)
    E_sensor = L_star * quantum_efficiency_
    return E_sensor


def total_number_of_incident_photon_per_second_per_area(lambda_interval_bottom_, lambda_interval_top_,
                                                        radiant_flux_, quantum_efficiency_):
    E_range = quad(sensor_photon_irradiance, lambda_interval_bottom_, lambda_interval_top_,
                            args=(radiant_flux_, quantum_efficiency_))
    return E_range


def flux_of_photons_intensity(E_range_, aperture_area_):
    phi_sensor = E_range_ * aperture_area_
    return phi_sensor

def flux_intensity_scaled(phi_sensor_, magnitude_):
    phi_st = phi_sensor_ * 10**(0.4*magnitude_)
    return phi_st

def photoelectrons_per_exposure_cauculator(E_range_pe_smm2_, magnitude_star_, exposuretime_sec_, diameter_telescope_mm2_):
    photoelectrons_per_exposure = E_range_pe_smm2_ * (1.0/(2.5**(magnitude_star_ - 0)) * exposuretime_sec_ * np.pi * (diameter_telescope_mm2_/2)**2)
    return photoelectrons_per_exposure    


In [3]:
# From Angel and Woolf 1998
def signal_to_noise_ratio(photoelectrons_per_second_signal, dark_current_noise, read_out_noise, diffuse_background):
    """
    This function calculated the signal to noise, using as input the count of photoelectrons per second for all of these
    signal, dark_current_noise, read_out_noise, and diffuse_background. From Angel and Woolf 1998
    :param photoelectrons_per_second_signal:
    :param dark_current_noise:
    :param read_out_noise:
    :param diffuse_background:
    :return: signal to noise ratio
    """
    # This is the formula found in the paper
    snr = photoelectrons_per_second_signal / np.sqrt(photoelectrons_per_second_signal + dark_current_noise +
                                                     read_out_noise + diffuse_background)

    return snr

## Assumptions

In [4]:
# Wavelength for passband:
# Assumption: H-band 
lambda_interval_bottom = 1300
lambda_interval_top = 1900

# temperature star
# Assumption: a M5 star (http://astro.vaporia.com/start/mclass.html)
temperature = 2800  # K

# sun's flux
flux_sun = 1361  # W/m^2

# aperture telescope
# Assumption: cumlus 18.5cm aperture
diameter = 185 ##mm
aperture_area = np.pi * (diameter/2)**2

# magnitude star 
# Assumption: generic magnitude
magnitude_star = 18.0

# exposure time
# # Assumption: I should compare with cumlus proposal values, which is 60s, but for now I will take 1.0s . just to have the values as photoelectrons/second
exposure = 1.0

radiant_flux, radiant_flux_error = radiant_flux_calculator(flux_sun_=flux_sun,
                                                            lambda_interval_bottom_=lambda_interval_bottom,
                                                            lambda_interval_top_= lambda_interval_top,
                                                            temperature_=temperature)

print('F_star [W/m^2]: ', radiant_flux)
E_range = total_number_of_incident_photon_per_second_per_area(lambda_interval_bottom_=lambda_interval_bottom*1e-9,
                                                              lambda_interval_top_=lambda_interval_top*1e-9,
                                                              radiant_flux_=radiant_flux,
                                                              quantum_efficiency_=0.45)

print('E_range [photoelectrons / s m^2]: ', E_range[0])


photoelectrons = photoelectrons_per_exposure_cauculator(E_range[0], magnitude_star, exposure, diameter)
print(f'We see {photoelectrons} [photoelectrons/second] in the H filter, from a Star mag {magnitude_star} and temperature: {temperature} \n using a {diameter/2} mm radius telescope')

F_star [W/m^2]:  3.1969070305431132e-09
E_range [photoelectrons / s m^2]:  6952.436459073605
We see 12.84251880871054 [photoelectrons/second] in the H filter, from a Star mag 18.0 and temperature: 2800 
 using a 92.5 mm radius telescope


In [5]:
# Dark current 
# Assumptions: from cumlus proposal 
dark_current = 0.05 # (electrons/second)

# Read out noise
# Assumptions: from cumlus proposal 
read_noise_every_60_seconds = 18 # (electrons/60seconds)
read_out = 0.3 # (electrons/second)

# Diffuse Background
# Assumptions: from cumlus proposal
# PSF for 18cm optics, 1.4" in Hband 
total_sky_flux = 715 # (photons/60seconds)
total_sky_flux_etenue_qe = 547 # (photons/60seconds)
background = 9.11 # photons/second

snr = signal_to_noise_ratio(photoelectrons_per_second_signal=photoelectrons, dark_current_noise=dark_current, read_out_noise=read_out, diffuse_background=background)
print(f'S/N is {snr}. ')

S/N is 2.7194010024010358. 


In [6]:
print(f'Assumptions:\nHband {lambda_interval_bottom}-{lambda_interval_top} nm')
print(f'Temperature: {temperature} K')
print(f'Suns flux: {flux_sun} W/m^2')
print(f'Aperture diameter: {diameter} mm')
print(f'Magnitude star: {magnitude_star}')
print('----------------------------------')
print(f'We see {photoelectrons} [photoelectrons/second] in the H filter, from a Star mag {magnitude_star} and temperature: {temperature}',
      f'\nusing a {diameter/2} mm radius telescope')

print(f'\nDark current: {dark_current} electrons/second')
print(f'Read out noise: {read_out} electrons/second')
print(f'Diffuse Background: {background} photons/second')
print('----------------------------------')
print(f'We have a S/N equals to {snr}. ')

Assumptions:
Hband 1300-1900 nm
Temperature: 2800 K
Suns flux: 1361 W/m^2
Aperture diameter: 185 mm
Magnitude star: 18.0
----------------------------------
We see 12.84251880871054 [photoelectrons/second] in the H filter, from a Star mag 18.0 and temperature: 2800 
using a 92.5 mm radius telescope

Dark current: 0.05 electrons/second
Read out noise: 0.3 electrons/second
Diffuse Background: 9.11 photons/second
----------------------------------
We have a S/N equals to 2.7194010024010358. 


## TODO:
- QE that varies with the wavelength (instead of a fix value) so we can use the camera values
- QE for  H4RG: 
Mosby Jr, Gregory, et al. "Properties and characteristics of the WFIRST H4RG-10 detectors."
- QE for Commercial camera:
Goldeye G-130 TEC1
